[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read dataset with extra params sep='\t', encoding="latin-1"
df = pd.read_csv("./DataSet/labeledTrainData.csv", sep="\t", encoding="latin-1")
df.sample(10)

,id,sentiment,review
14420,1458_9,1,"Sharp, well-made documentary focusing on Mardi..."
8164,4047_1,0,"*****WARNING, MAY CONTAIN SPOILERS WHICH WILL ..."
7820,9178_10,1,"I really have to say, this was always a favori..."
22664,4307_1,0,Seemingly intended to be a thriller of a movie...
20944,3247_3,0,I was seriously looking forward to seeing this...
18318,6504_10,1,My wife and I have watched this whole series a...
14394,6970_8,1,This is one of those Film's/pilot that if you ...
24771,9984_9,1,I was surprised at just how much I enjoyed thi...
146,9934_3,0,"As a casual listener of the Rolling Stones, I ..."
2149,6499_2,0,What a muddled mess. I saw this with a friend ...


## 2. Preprocessing

In [3]:
# stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    
    # Save emoticons for later appending
    # Your code here
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [ ]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split

X = df['review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [ ]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

clf = Pipeline([('vect',tfidf),
                ('clf',LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


## 4. Evaluate Model

In [ ]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import classification_report, confusion_matrix

predict = clf.predict(X_test)
print("confusion matrix:\n",confusion_matrix(y_test,predict))
print("classification report:\n",classification_report(y_test,predict))

## 5. Export Model 

In [ ]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open(os.path.join('DataSet', 'logisticRegression.pkl'),'wb'), protocol=4)